<a href="https://colab.research.google.com/github/yschimpf/feedback_alignment/blob/main/backpropagtion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np

In [5]:
class NeuralNetwork(object):
  #layers include input and output layer, activiation allows for an activation
  #to allow for f.e. softmax in case of application
  def __init__(self, layers = [2,10,1], activations = ['sigmoid','identity'], loss = 'MSE'):
    assert(len(layers) == len(activations)+1)
    self.layers = layers
    self.activations = activations
    self.loss = loss
    self.weights = []
    self.biases = []
    #fill the matrices with random values
    for i in range (len(layers)-1):
      self.weights.append(np.random.randn(layers[i+1], layers[i]))
      self.biases.append(np.random.randn(layers[i+1], layers[i]))

    # computation of all unit inputs and activations fpr all layers
    def feedfoward(self, x):
      #first layer has no activation function
      v = np.copy(x)
      z_s = []
      v_s = [v]
      #for all other leayers compute the weighted inputs of last layer units
      #and apply activations. Last layer has activation to f.e. apply softmax
      for i in range(len(self.weights)):
        activation = self.getActivationFunction(self.activation[i])
        z_s.append(self.weights[i].dot(v)+self.biases[i])
        v = activation(z_s[-1])
        v_s.append(v)
      return (z_s, v_s)


    #gets labels and feedforward data
    #returns derivatives w.r.t. weight matrices
    def backpropagation(self, y, z_s, v_s):
      #derivatives with respect to the weighs
      dw = []
      #error backprogated through the layers i.e. dL/dz i.e. error
      #in terms of inputs to the respective layer
      deltas = [None]*len(self.weights)
      #error in terms of inputs of last layer
      deltas[-1] = (self.getDerivativeLoss(self.loss)(v_s[-1], y)) * (self.getDerivativeActivation(self.activations[-1])(z_s[-1]))
      #actual backpropagation
      for i in reversed(range(len(deltas)-1)):
        #compute dL/dz for each layer that is the loss in terms of the inputs of the activation
        deltas[i] = self.weights[i+1].T.dot(deltas[i+1])*(self.getDerivativeActivation(self.activations[i])(z_s[i]))
      batch_size = y.shape[1]
      #compute dL/dw for all weight matrices
      dw = [d.dot(v_s[i].T)/float(batch_size) for i,d in enumerate(deltas)]
      #compute dL/db for all biases
      db = [d.dot(np.ones((batch_size,1)))/float(batch_size) for i,d in enumerate(deltas)]
      return dw, db

    #train netowrk on the training data
    def train(self, x, y, batch_size=10, epochs=100, lr = 0.01):
      for e in range (epochs):
        i = 0
        while(i < len(y)):
          x_batch = x[i:i+batch_size]
          y_batch = y[i:i+batch_size]
          i = i+batch_size
          z_s, v_s = self.feedforward(x_batch)
          dw, db = self.backpropagation(y_batch, z_s, v_s)
          self.weights = [w - lr*dweight for w,dweight in zip(self.weights, dw)]
          self.biases = [b - lr*dbias for b,dbias in zip(self.biases, db)]
          print("loss ={}".format(np.linalg.norm(v_s[-1]-y_batch)))

    #definition of nonlinearities for units
    @staticmethod
    def getActivationFunction(name):
      if(name == 'sigmoid'):
        return lambda x : np.exp(x)/(1+np.exp(x))
      elif(name == 'identity'):
        return lambda x : x
      elif(name == 'relu'):
        def relu(x):
          y = np.copy(x)
          if(y<0):
            return 0
          else:
            return y
        return relu
      else:
        print('Unknown activation function. Identity is used')
        return lambda x:x

    #derivative of activation
    @staticmethod
    def getDerivativeActivation(name):
      if(name == 'sigmoid'):
        sigmoid = lambda x : np.exp(x)/(1+np.exp(x))
        return lambda x : sigmoid(x)*(1-sigmoid(x))
      elif(name == 'identity'):
        return lambda x : 1
      elif(name == 'relu'):
        def drelu(x):
          y = np.copy(x)
          if(y >= 0): y = 1
          else: y = 0
          return y
        return drelu
      else:
        print('Unknown activation, identity was used instead')
        return lambda x:1


    #derivative of loss
    @staticmethod
    def getDerivativeLoss(name):
      #compute derivative of MSE for formula L = 1/2 sum_{i}(f_i - y_i)^2 w.r.t. f_i
      #where i sums over the samples, f_i is the network output and y-i the label
      if(name == 'MSE'):
        return lambda f, y : (f-y)
      #without a specified loss computing a loss does not really make sense
      else:
        print('Unknown loss, cannot make useful computations')
        exit()


In [5]:
#testing stuff
arr = np.array([1,2,3,4])
print(arr[-1])

4
